<a href="https://colab.research.google.com/github/Smile17/GoogleCollaboration/blob/main/ChatBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pymorphy2

In [ ]:
import nltk
import random
import json
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from pymorphy2 import MorphAnalyzer
import pickle

In [ ]:
with open('/content/BOT_CONFIG.json') as f:
  BOT_CONFIG = json.load(f)
BOT_CONFIG

{'intents': {' compliment': {'examples': ['ты классный',
    'ты веселый',
    'ты супер',
    'ты молодец',
    'ты общительный'],
   'responses': ['спасибо!',
    'мне приятно!',
    'ты тоже ничего ;-) !',
    'ты тоже ) ']},
  'ability': {'examples': ['что ты умеешь',
    'расскажи о себе',
    'что ты можешь'],
   'responses': ['я бот №4372, пока ничего не умею :(',
    'я умный бот, но в настоящее время я не умею выполнять никакие команды',
    'пока ничего не могу, но учусь']},
  'accord': {'examples': ['да', 'хорошо', 'конечно'],
   'responses': ['отлично', 'замечательно', 'чудесно']},
  'action': {'examples': ['что делаешь', 'чем занимаешься'],
   'responses': ['думаю о захвате человечества\nа вы чем заняты?',
    'возвожу в 10 степень пятизначные числа.\nа вы?',
    'обучаюсь на комментариях во вконтакте.\nчем вы заняты?']},
  'actions': {'examples': ['расскажи историю',
    'расскажи что-нибудь интересное',
    'расскажи анекдот',
    'расскажи что-нибудь смешное',
    'анек

## День 1

In [ ]:
# BOT_CONFIG = {
#     'intents': {
#         'hello': {
#             'examples': ['Привет!', 'Хэлло', 'Хей'],
#             'responses': ['Добрый день', 'Добрый вечер', 'Здравствуйте']
#         },
#         'bye': {
#             'examples': ['Пока...', 'Увидимся!', 'До скорого'],
#             'responses': ['До свиданья', 'До встречи', 'Хорошего дня!']
#         }
#     }
# }

In [ ]:
def clean(text):
  cleaned_text = ''
  for char in text.lower():
    if char in 'абвгдеёжзийклмнопрстуфхцчшщъыьэюя ':
      cleaned_text += char #cleaned_text = cleaned_text + char
  return cleaned_text

def get_intent(in_text):
  for intent in BOT_CONFIG['intents'].keys():
    for example in BOT_CONFIG['intents'][intent]['examples']:
      text1 = clean(example)
      text2 = clean(in_text)
      if nltk.edit_distance(text1, text2) / max(len(text1), len(text2)) < 0.4:
        return intent
  return 'Не удалось определить интент'

In [ ]:
def bot(in_text):
  intent = get_intent(in_text)

  if intent == 'Не удалось определить интент':
    return 'Извите, я ничего не понял'
  else:
    return random.choice(BOT_CONFIG['intents'][intent]['responses'])

In [ ]:
in_text = ''
while in_text != 'Выход':
  in_text = input()
  out_text = bot(in_text)
  print(out_text)

фвіф
Извите, я ничего не понял
привет
добрый день
как дела
бывало лучше
виход
до скорых встреч!
Выход
до скорых встреч!


## Обучение модели

In [ ]:
X, y = [], []

for intent in BOT_CONFIG['intents']:
  for example in BOT_CONFIG['intents'][intent]['examples']:
    X.append(example)
    y.append(intent)

In [ ]:
morph = MorphAnalyzer()

def lemmatize(text):
  lemmatized_text = []
  for word in text.lower().split():
    lemmatized_text.append(morph.parse(word)[0].normal_form)
  return ' '.join(lemmatized_text)

2021-06-27 11:24:06,003 - pymorphy2.opencorpora_dict.wrapper - INFO - Loading dictionaries from /usr/local/lib/python3.7/dist-packages/pymorphy2_dicts_ru/data
2021-06-27 11:24:06,147 - pymorphy2.opencorpora_dict.wrapper - INFO - format: 2.4, revision: 417127, updated: 2020-10-11T15:05:51.070345


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [ ]:
X_train = [lemmatize(text) for text in X_train]
X_test = [lemmatize(text) for text in X_test]

In [ ]:
vectorizer = CountVectorizer(analyzer='word', ngram_range=(1,3))
vectorizer.fit(X_train)
X_vectors = vectorizer.transform(X_train)
len(vectorizer.get_feature_names())

1254

In [ ]:
classifier = LogisticRegression(max_iter=200)
classifier.fit(X_vectors, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=200,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
from sklearn import svm
classifier = svm.LinearSVC()
classifier.fit(X_vectors, y_train)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [ ]:
from sklearn.linear_model import SGDClassifier
classifier = SGDClassifier(max_iter=3)
classifier.fit(X_vectors, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_stochastic_gradient.py:557: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)


SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='hinge', max_iter=3,
              n_iter_no_change=5, n_jobs=None, penalty='l2', power_t=0.5,
              random_state=None, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

In [ ]:
classifier.score(X_vectors, y_train)

0.7496206373292867

In [ ]:
classifier.score(vectorizer.transform(X_test), y_test)

0.14864864864864866

In [ ]:
with open('/content/our_ml_model.pickle', 'wb') as f:
  pickle.dump(classifier, f)

with open('/content/our_vectorizer.pickle', 'wb') as f:
  pickle.dump(vectorizer, f)

In [ ]:
with open('/content/our_ml_model.pickle', 'rb') as f:
  loaded_classifier = pickle.load(f)

with open('/content/our_vectorizer.pickle', 'rb') as f:
  loaded_vectorizer = pickle.load(f)

In [ ]:
loaded_classifier.predict(loaded_vectorizer.transform(['как дела?']))

array(['how are you?'], dtype='<U29')

## Тестирование бота

In [ ]:
def get_intent_by_ml_model(in_text):
  return classifier.predict(vectorizer.transform([in_text]))[0]

def bot(in_text):
  intent = get_intent_by_ml_model(in_text)

  if intent == 'Не удалось определить интент':
    return 'Извите, я ничего не понял'
  else:
    return random.choice(BOT_CONFIG['intents'][intent]['responses'])

In [ ]:
# in_text = ''
# while in_text != 'Выход':
#   in_text = input()
#   out_text = bot(in_text)
#   print(out_text)

## Подключение чат-бота к Telegramm

In [ ]:
!pip install python-telegram-bot --upgrade

Requirement already up-to-date: python-telegram-bot in /usr/local/lib/python3.7/dist-packages (13.6)


In [ ]:
import logging

from telegram import Update, ForceReply
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters, CallbackContext

# Enable logging
logging.basicConfig(
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', level=logging.INFO
)

logger = logging.getLogger(__name__)


# Define a few command handlers. These usually take the two arguments update and
# context.
def start(update: Update, context: CallbackContext) -> None:
    """Send a message when the command /start is issued."""
    user = update.effective_user
    update.message.reply_markdown_v2(
        fr'Hi {user.mention_markdown_v2()}\!',
        reply_markup=ForceReply(selective=True),
    )


def help_command(update: Update, context: CallbackContext) -> None:
    """Send a message when the command /help is issued."""
    update.message.reply_text('Help!')


def echo(update: Update, context: CallbackContext) -> None:
    update.message.reply_text(bot(update.message.text))


def main() -> None:
    """Start the bot."""
    # Create the Updater and pass it your bot's token.
    updater = Updater("1720163957:AAGoeoNcAB6Fr3AScRXLnKeuskQeFsZ6BDs")

    # Get the dispatcher to register handlers
    dispatcher = updater.dispatcher

    # on different commands - answer in Telegram
    dispatcher.add_handler(CommandHandler("start", start))
    dispatcher.add_handler(CommandHandler("help", help_command))

    # on non command i.e message - echo the message on Telegram
    dispatcher.add_handler(MessageHandler(Filters.text & ~Filters.command, echo))

    # Start the Bot
    updater.start_polling()

    # Run the bot until you press Ctrl-C or the process receives SIGINT,
    # SIGTERM or SIGABRT. This should be used most of the time, since
    # start_polling() is non-blocking and will stop the bot gracefully.
    updater.idle()

In [ ]:
main()

2021-06-27 11:33:28,384 - apscheduler.scheduler - INFO - Scheduler started
